# Détection Pierre-feuille-ciseaux 

Pierre-papier-ciseaux est un jeu effectué avec les mains.

**Objectif : créer un modèle permettant de classifier une image d'une main afin de savoir si la personne joue une pierre, une feuille ou une paire de ciseaux**
- Chargement jeu de données CIFAR10
- Affichage des données
- Data Augmentation
- Création d'un modèle CNN
- Affichage de l'accuracy et du loss
- Visualisation des représentations intermédiaires
- Ajout d'un callback (earlyStopping)


## ◢ 1 Chargement des données

Commençons par télécharger le jeu de données d'entrainement et de test. 

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip
  
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip \
    -O /tmp/rps-test-set.zip

--2023-02-02 14:55:54--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 142.251.31.128, 142.250.153.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-02-02 14:55:55 ERROR 404: Not Found.

--2023-02-02 14:55:55--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 142.251.31.128, 142.250.153.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-02-02 14:55:55 ERROR 404: Not Found.



Les fichiers seront dézippés dans le repertoire tmp. Si vous utilisez Google Calab, vous pouvez accéder à l'arborescence de fichiers via le menu fichier à gauche de la fenêtre.

In [2]:
import os
import zipfile

local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

local_zip = '/tmp/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

BadZipFile: ignored

In [ ]:
rock_dir = os.path.join('/tmp/rps/rock')
#Définir les chemins pour les images représentant des feuilles et des ciseaux
paper_dir = os.path.join('/tmp/rps/paper')
scissors_dir = os.path.join('/tmp/rps/scissors')

print('total training rock images:', len(os.listdir(rock_dir)))
#Afficher du nombre d'images représentant des feuilles et des ciseaux
print('total training paper images:', len(os.listdir(paper_dir)))
print('total training scissors images:', len(os.listdir(scissors_dir)))

rock_files = os.listdir(rock_dir)
print(rock_files[:10])

#Obtenir la liste des images représentant des feuilles et des ciseaux
paper_files = os.listdir(paper_dir)
print(paper_files[:10])

scissors_files = os.listdir(scissors_dir)
print(scissors_files[:10])

## ◢ 2 Affichage des données

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2


next_rock = [os.path.join(rock_dir, fname) 
                for fname in rock_files[pic_index-2:pic_index]]
next_paper = [os.path.join(paper_dir, fname) 
                for fname in paper_files[pic_index-2:pic_index]]
next_scissors = [os.path.join(scissors_dir, fname) 
                for fname in scissors_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_rock+next_paper+next_scissors):
  img = mpimg.imread(img_path)
  #Affichage de l'image via la librairie matplot
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

## ◢ 3 Data Augmentation

TensorFlow (Keras) propose une API de preprocessing pour les images. 

Cette API permet notamment de parcourir la structure d'un répertoire contenant des images, de les charger et de renvoyer les images (tableaux de pixels) et la sortie/le label (nom du répertoire).

Elle peut également augmenter le jeu de données en appliquant aléatoirement différentes transformations.

La Data Augmentation est une technique qui peut être utilisée pour augmenter artificiellement la taille d'un ensemble de données d'entrainement en créant des versions modifiées des images du dataset.

[Documentation TF d'ImageGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

In [ ]:
TRAINING_DIR = "/tmp/rps/"

#Création d'un ImageDataGenerator avec
# en complément des augmentations déjà listées : un rescale de 1./255,  
# un angle de rotation allant de 0 à 40, un retournement horizontal
training_datagen = ImageDataGenerator(
    rescale = 1./255,
	rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

#Itération sur le repértoire d'entrainement avec un batch size de 128
train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
    batch_size=128
)

#Création d'un ImageDataGenerator avec
# un rescale de 1./255. Attention, sur le jeu de test, nous n'avons 
# pas besoin d'augmenter le jeu de données. 
# Nous validerons le modèle sur un jeu réel.
VALIDATION_DIR = "/tmp/rps-test-set/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

#Itération sur le repértoire de tests en utilisant le même 
# paramètrage que pour le jeu d'entrainement
validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
    batch_size=128
)

## ◢ 4 Création d'un modèle CNN

In [ ]:
#Création d'un réseau CNN
model = tf.keras.models.Sequential([                   
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


model.summary()



In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=25, steps_per_epoch=20, 
                    validation_data = validation_generator, 
                    verbose = 1, validation_steps=3
                   )

model.save("rps.h5")

## ◢ 5 Affichage de l'accuracy et du loss

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

## ◢ 6 Visualisation des représentations intermédiaires

In [ ]:
import numpy as np
import random
from   tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers]

visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

rock_img_files = [os.path.join(rock_dir, f) for f in rock_files]
paper_img_files = [os.path.join(paper_dir, f) for f in paper_files]
scissors_img_files = [os.path.join(scissors_dir, f) for f in scissors_files]

img_path = random.choice(rock_img_files +  paper_img_files + scissors_img_files)

print(img_path)
img = load_img(img_path, target_size=(150, 150))  # this is a PIL image

x   = img_to_array(img)                           # Numpy array with shape (150, 150, 3)
x   = x.reshape((1,) + x.shape)                   # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255.0

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# -----------------------------------------------------------------------
# Now let's display our representations
# -----------------------------------------------------------------------
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:

    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display of the grid
    #-----------------

    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' ) 

## ◢ 7 Ajout d'un callback pour arrêter l'entrainement dès lors qu'un seuil est atteint.

Ici nous arrêtons l'entrainement dès que l'accuracy est > à 85%.

Une fois ce callback défini, relancez l'entrainement (fit) du modèle en ajoutant ce callback et notez le comportement.

[Documentation Callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback)

[Tutoriel Callback](https://www.tensorflow.org/guide/keras/custom_callback)

In [ ]:
#Nous allons ici relancer l'apprentissage en ajoutant ce callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.85):
      print("\nReached 85% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
from PIL import Image
import numpy as np
img = Image.open('ma-main.jpg')

In [ ]:
np.expand_dims(np.array(img), axis=0).shape

In [ ]:
img = np.expand_dims(np.array(img), axis=0)

img.shape

In [ ]:
model.predict(img)

In [ ]:
my_img = mpimg.imread('ma-main.jpg')

my_img2 = np.expand_dims(my_img, axis=0)

model.predict(my_img2)